# The Product Pricer With Linear Regression

Traditional machine learning models that estimate how much a product costs based on extracted numeric features, such as weight, brand, text length, and popularity rank — rather than raw text descriptions.

## Purpose

To demonstrate multiple regression models that can predict prices from structured data, using Feature Linear Regression, Bag of Words, Word2Vec & Linear Regression, word2vec & SVR(Support Vector Regression), word2vec & Random Forest, and carry out performance comparison across different ML models

### Baseline Models

In [ ]:
!pip install pandas numpy scikit-learn gensim huggingface-hub

In [ ]:
### General Imports
import os
import math
import json
import random
from dotenv import load_dotenv
import matplotlib.pyplot as plt 
import pickle 
from collections import Counter
from huggingface_hub import login

In [ ]:
### Imports For Traditional Machine Learning

import pandas as pd 
import numpy as np 
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

In [ ]:
### For NLP related machine learning

from sklearn.feature_extraction.text import CountVectorizer 
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

In [ ]:
### More of machine learning related ones
from sklearn.svm import LinearSVR 
from sklearn.ensemble import RandomForestRegressor 

In [ ]:
### Internal Classes

from items import Item

In [ ]:
### Constants -- used for printing to stdout in color

GREEN = "\033[92m"
YELLOW = "\033[93m"
RED = "\033[91m"
RESET = "\033[0m"
COLOR_MAP = {"green": GREEN, "orange": YELLOW, "red": RED} 

In [ ]:
### Environment 

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")
HF_TOKEN_KEY = os.getenv("HF_TOKEN")

login(HF_TOKEN_KEY)


In [ ]:
%matplotlib inline

## Loading the pkl files

In [ ]:
with open("train.pkl", "rb") as f:
    train = pickle.load(f)

with open("test.pkl", "rb") as f:
    test = pickle.load(f)

In [ ]:
print(test[0].test_prompt())

In [ ]:
print(test[0].price)

## Tester Class

In [ ]:
class Tester: 

    def __init__(self, predictor, title=None, data=test, size=250): 
        self.predictor = predictor
        self.data = data
        ### Conditionally used for either the text report or chart's title
        self.title = title or predictor.__name__.replace("_", " ").title()
        self.size = size
        self.truths = []
        self.guesses = []
        self.errors = []
        self.sles = []
        self.colors = []

    def color_for(self, error, truth): 
        if error < 20 or error / truth < 0.2:
            return "green"
        elif error < 80 or error / truth < 0.4: 
            return "orange"
        else:
            return "red"

    def run_datapoint(self, i): 
        datapoint = self.data[i]
        guess = self.predictor(datapoint)
        truth = datapoint.price
        error = abs(guess - truth)
        log_error = math.log(truth + 1) - math.log(guess + 1)
        sle = log_error ** 2
        color = self.color_for(error, truth)
        title = datapoint.title if len(datapoint.title) < 40 else datapoint.title[:40] + "..."
        self.truths.append(truth)
        self.guesses.append(guess)
        self.errors.append(error) 
        self.sles.append(sle) 
        self.colors.append(color) 
        print(f"{COLOR_MAP[color]}{i + 1}: Guess: ${guess:,.2f} | Truth: ${truth:,.2f} | Error: {error:,.2f} | SLE: {sle:,.2f} Item: {title}{RESET}")

    def chart(self, title): 
        max_error = max(self.errors) 
        plt.figure(figsize=(12, 8))
        max_val = max(max(self.truths), max(self.guesses))
        plt.plot([0, max_val], [0, max_val], color="deepskyblue", lw=2, alpha=0.6)
        plt.scatter(self.truths, self.guesses, s=3, c=self.colors)
        plt.xlabel('Ground Truth')
        plt.ylabel('Model Estimate')
        plt.xlim(0, max_val)
        plt.ylim(0, max_val) 
        plt.title(title)
        plt.show()

    def report(self): 
        average_error = sum(self.errors) / self.size
        rmsle = math.sqrt(sum(self.sles) / self.size)
        hits = sum(1 for color in self.colors if color == "green")
        title = f"{self.title} Error=${average_error:,.2f} RMSLE={rmsle:,.2f} Hits={hits / self.size * 100:.1f}%"
        self.chart(title)

    def run(self):
        for i in range(self.size): 
            self.run_datapoint(i) 
        self.report()

    @classmethod
    def test(cls, function):
        cls(function).run()


In [ ]:
### Simulator function

def random_pricer(item): 
    return random.randrange(1, 1000)

In [ ]:
random.seed(42)

### Run Tester with a simulating guess predictor
Tester.test(random_pricer)

In [ ]:
### Another test -- with average price from the train data

training_prices = [item.price for item in train]
training_avr_price = sum(training_prices) / len(training_prices)

def constant_pricer(item): 
    return training_avr_price 

In [ ]:
Tester.test(constant_pricer)

## Feature Engineering

Demonstrate feature engineering using product weights, brand, and other fields 
to.

In [ ]:
train[0].details

In [ ]:
### Create a new "features" field on items, 
### and populate it with json parsed from the details dict

for item in train: 
    item.features = json.loads(item.details)

for item in test: 
    item.features = json.loads(item.details)

In [ ]:
print(train[0].features.keys())

In [ ]:
### Look at 20 most common features in training set

feature_count = Counter()

for item in train: 
    for f in item.features.keys(): 
        feature_count[f] += 1

feature_count.most_common(20)

### Product Weights

In [ ]:
# Some janky code to pluck out the Item Weight

def get_weight(item):
    weight_str = item.features.get('Item Weight')
    if weight_str:
        parts = weight_str.split(' ')
        amount = float(parts[0])
        unit = parts[1].lower()
        if unit=="pounds":
            return amount
        elif unit=="ounces":
            return amount / 16
        elif unit=="grams":
            return amount / 453.592
        elif unit=="milligrams":
            return amount / 453592
        elif unit=="kilograms":
            return amount / 0.453592
        elif unit=="hundredths" and parts[2].lower()=="pounds":
            return amount / 100
        else:
            print(weight_str)
    return None

In [ ]:
weights = [get_weight(t) for t in test]

### Filter out None values (if w)
weights = [w for w in weights if w]

In [ ]:
average_weight = sum(weights) / len(weights)
print(f"Avr Weight: {average_weight:.2f}")

In [ ]:
### Set default weight for items with None value for weights
def get_defualt_weight(item): 
    weight = get_weight(item)
    return weight or average_weight

In [ ]:
w = get_weight(train[1000])
print(w)

### Best Seller Ranks 

In [ ]:
def get_rank(item): 
    """
    Get an average rank per item out of multiple ranks across different categories.
    """
    ### A product has ranks across multiple categories in Amazon data
    ranks_dict = item.features.get("Best Sellers Rank")
    if ranks_dict:
        ranks = ranks_dict.values()
        avr_rank = sum(ranks) / len(ranks)
        return avr_rank
        
    return None
    
    

In [ ]:
get_rank(train[100])

In [ ]:
ranks = [get_rank(t) for t in train]
ranks = [r for r in ranks if r]

average_rank = sum(ranks) / len(ranks)
print(f"Average Rank = {average_rank:,.2f}")

In [ ]:
def get_default_rank(item): 
    rank = get_rank(item)
    return rank or average_rank

In [ ]:
def get_text_length(item): 
    return len(item.prompt)

In [ ]:
### Look at 40 most common brands

brand_counts = Counter()

for t in train: 
    brand = t.features.get("Brand")
    if brand: 
        brand_counts[brand] += 1 

print(brand_counts.most_common(40))

In [ ]:
TOP_ELECTRONICS_BRANDS = ["hp", "dell", "lenovo", "samsung", "asus", "sony", "canon", "apple", "intel"]

def is_top_electronics_brand(item):
    brand = item.features.get("Brand")
    return brand and brand.lower() in TOP_ELECTRONICS_BRANDS

print(is_top_electronics_brand(train[2300]))

### Result: get_features()

In [ ]:
def get_features(item): 
    """
    Return weight, rank, text_length, and whether it's one of top electronics brands in a dictionary form
    """
    return {
        "weight": get_defualt_weight(item), 
        "rank": get_default_rank(item), 
        "text_length": get_text_length(item), 
        "is_top_electronics_brand": 1 if is_top_electronics_brand(item) else 0 
    }

In [ ]:
get_features(train[0])

## Linear Regression with Features

In [ ]:
# A utility function to convert the features(dict) into a pandas dataframe

def list_to_dataframe(items):
    features = [get_features(item) for item in items]
    df = pd.DataFrame(features)
    ### Add price label
    df["price"] = [item.price for item in items]

    return df

train_df = list_to_dataframe(train)
test_df = list_to_dataframe(test[:250])

In [ ]:
print(test_df[:10])

In [ ]:
### Traditional Linear Regression

np.random.seed(42)

### Separate features and target 
feature_columns = ["weight", "rank", "text_length", "is_top_electronics_brand"]

X_train = train_df[feature_columns]
y_train = train_df["price"]

X_test = test_df[feature_columns]
y_test = test_df["price"]

### Train a Linear Regression
model = LinearRegression()
model.fit(X_train, y_train)

print(model.coef_)

### Print pairs of feature_columns and coefficients 
print("\n|==Metrics Report==|\n")
for features, coef in zip(feature_columns, model.coef_): 
    print(f"{features}: {coef}")

### Intercept rate
print(f"Intercept: {model.intercept_}")


### Predict the test set and evaluate (with error metrics)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squred Error: {mse}")
print(f"R-Squared Score: {r2}")

### Function to predict price for a new item

def linear_regression_pricer(item):
    features = get_features(item)
    features_df = pd.DataFrame([features])
    
    return model.predict(features_df)[0]

In [ ]:
Tester.test(linear_regression_pricer)

## Bag of Words (NLP)

Using the Bag of Words (BoW) model, the results showed more than twice the improvement compared to the feature engineering approach.
(hits: 10% TO 24%)

In [ ]:
### X and y elements
prices = np.array([float(item.price) for item in train])
documents = [item.test_prompt() for item in train]

In [ ]:
# np.random.seed(42)
vectorizer = CountVectorizer(max_features=1000, stop_words="english")
X = vectorizer.fit_transform(documents)
regressor = LinearRegression()
regressor.fit(X, prices)

In [ ]:
def bow_lr_pricer(item): 
    x = vectorizer.transform([item.test_prompt()])
    return max(regressor.predict(x)[0], 0)

In [ ]:
Tester.test(bow_lr_pricer)

## word2vec model

In [ ]:
### word2vec model implemented in gensim NLP library

np.random.seed(42)

### Prepare the documents: (test_prompt used in the BoW model above)
processed_docs = [simple_preprocess(doc) for doc in documents]

### Train Word2Vec model: 
w2v_model = Word2Vec(sentences=processed_docs, vector_size=400, window=5, min_count=1, workers=8)

In [ ]:
### Note: This step of averaging vectors across the document is a weakness in this approach

def document_vector(doc): 
    doc_words = simple_preprocess(doc) ### split words and store
    ### convert the split words into numeric representations (vectors)
    word_vectors = [w2v_model.wv[word] for word in doc_words if word in w2v_model.wv]
    
    return np.mean(word_vectors, axis=0) if word_vectors else np.zeros(w2v_model.vector_size)

### Create feature matrix: 
X_w2v = np.array([document_vector(doc) for doc in documents])

In [ ]:
# print(w2v_model.wv["computer"])
print(w2v_model.wv["computer"].shape)

In [ ]:
### Run Linear Regression on word2vec 

word2vec_lr_regressor = LinearRegression()
word2vec_lr_regressor.fit(X_w2v, prices)

In [ ]:
def word2vec_lr_pricer(item): 
    doc = item.test_prompt()
    doc_vector = document_vector(doc)
    return max(0, word2vec_lr_regressor.predict([doc_vector])[0])

In [ ]:
Tester.test(word2vec_lr_pricer)

## Support Vector Machines 

In [52]:
np.random.seed(42)
svr_regressor = LinearSVR()

svr_regressor.fit(X_w2v, prices)

,epsilon,0.0
,tol,0.0001
,C,1.0
,loss,'epsilon_insensitive'
,fit_intercept,True
,intercept_scaling,1.0
,dual,'auto'
,verbose,0
,random_state,None
,max_iter,1000


In [54]:
# function that receives an item argument 
# take out a test prompt from item
# vectorize it
# predict using svr_regressor instance

def svr_pricer(item): 
    np.random.seed(42)
    doc = item.test_prompt()
    doc_vector = document_vector(doc)
    return max(float(svr_regressor.predict([doc_vector])[0]), 0)